In [7]:
from transformers import pipeline
from nltk import sent_tokenize
import nltk
import torch

In [6]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anubh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

###LOAD MODEL

In [8]:
model_name="facebook/bart-large-mnli"
device=0 if torch.cuda.is_available() else "cpu"

In [13]:
def load_model (device):
          theme_classifier=pipeline("zero-shot-classification",model=model_name,device=device)
          return theme_classifier

In [14]:
theme_classifier=load_model(device)

RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.